In [1]:
import pandas as pd
import numpy as np
import time
import re

### Key.csv File

In [114]:
# key.csv 파일에서 분석 가능한 내용
# 1) 어떤 기지국(station)이 가장 많이 사용되었는지
# 2) 어떤 기지국(station)이 가장 적게 사용되었는지
# 따라서, 어떤 기주국(station)의 영향력이 가장 셀지 적을지 볼 수 있음
data_key = pd.read_csv('key.csv')
data_key

,store_nbr,station_nbr
0,1,1
1,2,14
2,3,7
3,4,9
4,5,12
5,6,14
6,7,6
7,8,4
8,9,17
9,10,12


In [116]:
# 17번 기지국은 6개 Store에서 기지국으로 사용함
# 13번 기지국은 5개 Store에서 기지국으로 사용함
# 12,14번 기지국은 4개 Store에서 기지국으로 사용함
# 3,7,10번 기지국은 3개 Store에서 기지국으로 사용함
# 6,9,11,16번 기지국은 2개 Store에서 기지국으로 사용함
# 1,2,4,5,8,15,18,19,20번 기지국은 1개의 Store에서 기지국으로 사용함.

count_data_key = data_key.groupby('station_nbr').size().reset_index(name='counts')
count_data_key = count_data_key.sort_values(by=['counts'], ascending=False)
count_data_key.reset_index(drop=True, inplace=True)
count_data_key

,station_nbr,counts
0,17,6
1,13,5
2,12,4
3,14,4
4,3,3
5,7,3
6,10,3
7,11,2
8,6,2
9,9,2


### Weather.csv File

In [122]:
data_weather = pd.read_csv('weather.csv')
data_weather

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0
3,4,2012-01-01,63,47,55,4,28,43,10,0,0728,1742,,0.0,0.00,29.79,30.48,8.0,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
5,7,2012-01-01,50,33,42,M,26,35,23,0,-,-,,0.0,0.00,29.15,30.54,10.3,32,10.2
6,8,2012-01-01,66,45,M,M,34,46,M,M,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
7,9,2012-01-01,34,19,27,M,17,23,38,0,-,-,UP,M,T,29.34,30.09,22.8,30,22.5
8,10,2012-01-01,73,53,63,M,55,58,2,0,0723,1738,FG+ FG BR,M,0.00,30.16,30.19,5.1,24,5.5
9,11,2012-01-01,72,48,60,7,54,56,5,0,0724,1737,FG+ FG BR,0.0,0.00,30.15,30.18,4.6,23,4.8


In [121]:
data_weather['codesum'][0]

'RA FZFG BR'

In [166]:
# 비 or 눈이 온 날 --> 아직 부정확함. M or T 도 세부적으로 나눠야 함.
data_weather[(data_weather['snowfall'] != '0.0') | (data_weather['preciptotal'] != '0.00')]

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
6,8,2012-01-01,66,45,M,M,34,46,M,M,-,-,RA BR,M,0.00,30.05,M,11.0,36,10.9
7,9,2012-01-01,34,19,27,M,17,23,38,0,-,-,UP,M,T,29.34,30.09,22.8,30,22.5
8,10,2012-01-01,73,53,63,M,55,58,2,0,0723,1738,FG+ FG BR,M,0.00,30.16,30.19,5.1,24,5.5
11,13,2012-01-01,38,18,28,M,12,21,37,0,-,-,,M,0.00,24.60,30.59,2.5,16,6.1
13,15,2012-01-01,48,26,37,16,35,38,28,0,0718,1621,RA FG+ FZFG BR,0.0,0.09,29.53,29.89,2.5,17,3.8
14,16,2012-01-01,50,30,40,M,37,40,25,0,-,-,RA MIFG BR,M,T,29.80,29.89,4.2,21,6.8
17,19,2012-01-01,38,25,32,10,26,30,33,0,0723,1628,RA SN BR,0.5,0.12,29.06,29.79,14.6,29,15.6
19,1,2012-01-02,50,31,41,M,26,35,24,0,-,-,,M,0.01,29.44,29.62,9.8,24,10.3


In [168]:
# 비 or 눈이 오지 않은 날 --> 아직 부정확함. Only contains 0.0 and 0.00 . Need to find the M or T meannings
data_weather[(data_weather['snowfall'] == '0.0') & (data_weather['preciptotal'] == '0.00')]

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0
3,4,2012-01-01,63,47,55,4,28,43,10,0,0728,1742,,0.0,0.00,29.79,30.48,8.0,35,8.2
4,6,2012-01-01,63,34,49,0,31,43,16,0,0727,1742,,0.0,0.00,29.95,30.47,14.0,36,13.8
5,7,2012-01-01,50,33,42,M,26,35,23,0,-,-,,0.0,0.00,29.15,30.54,10.3,32,10.2
9,11,2012-01-01,72,48,60,7,54,56,5,0,0724,1737,FG+ FG BR,0.0,0.00,30.15,30.18,4.6,23,4.8
10,12,2012-01-01,72,48,60,M,54,57,5,0,-,-,FG+ FG BR,0.0,0.00,30.15,30.19,4.4,25,4.8
12,14,2012-01-01,50,34,42,5,25,35,23,0,0739,1729,,0.0,0.00,29.13,30.52,11.4,32,11.3
15,17,2012-01-01,56,38,47,M,23,37,18,0,-,-,,0.0,0.00,29.83,30.50,5.4,32,5.8
16,18,2012-01-01,59,40,50,4,28,40,15,0,0730,1732,,0.0,0.00,29.98,30.49,9.0,33,8.9
18,20,2012-01-01,59,41,50,M,28,40,15,0,-,-,,0.0,0.00,29.78,30.48,10.9,33,10.8


### Train.csv File

In [118]:
data_train = pd.read_csv('train.csv')
data_train

,date,store_nbr,item_nbr,units
0,2012-01-01,1,1,0
1,2012-01-01,1,2,0
2,2012-01-01,1,3,0
3,2012-01-01,1,4,0
4,2012-01-01,1,5,0
5,2012-01-01,1,6,0
6,2012-01-01,1,7,0
7,2012-01-01,1,8,0
8,2012-01-01,1,9,29
9,2012-01-01,1,10,0


In [148]:
# 상품별 판매량 순위를 알아보고, Product중에 판매가 안된 것이 있는지 확인해 보자.
data_train_ranks = data_train.groupby('item_nbr').agg('sum').reset_index()
data_train_ranks = data_train_ranks[['item_nbr', 'units']]
data_train_ranks

,item_nbr,units
0,1,778
1,2,3269
2,3,675
3,4,348
4,5,846662
5,6,41398
6,7,721
7,8,30345
8,9,916615
9,10,745


In [150]:
data_train_ranks.loc[0:50]

,item_nbr,units
0,1,778
1,2,3269
2,3,675
3,4,348
4,5,846662
5,6,41398
6,7,721
7,8,30345
8,9,916615
9,10,745


In [151]:
data_train_ranks.loc[51:100]

,item_nbr,units
51,52,2068
52,53,561
53,54,449
54,55,940
55,56,1608
56,57,458
57,58,1024
58,59,781
59,60,710
60,61,2076


In [153]:
data_train_ranks.loc[101:]

,item_nbr,units
101,102,31
102,103,241
103,104,656
104,105,637
105,106,139
106,107,534
107,108,2045
108,109,4208
109,110,1402
110,111,254


In [156]:
# 가장 많이 팔린 상품 별로, 내림 차순 정렬
data_train_ranks_decending = data_train_ranks.sort_values(by=['units'], ascending = False).reset_index(drop=True)
data_train_ranks_decending
# 이 DataFrame을 통해서 확인 가능 한 것은, 
# 어떤 Product가 가장 팔리기 때문에 영향력이 큰지
# 어떤 product가 적게 팔리기 때문에 영향력이 적은지

,item_nbr,units
0,45,1005111
1,9,916615
2,5,846662
3,44,577193
4,16,226772
5,68,172243
6,25,148253
7,37,125156
8,36,83865
9,48,76092


In [140]:
# 위를 확인 한 이유는, 111개의 Product중에 팔리지 않은 상품이 있는지 
# 없는 지를 판별하기 위함
# 위의 data를 본 결과, 111개 모든 product는 판매 되었다는 것을 확인 함.

### Test.csv

In [123]:
data_test = pd.read_csv('test.csv')
data_test

,date,store_nbr,item_nbr
0,2013-04-01,2,1
1,2013-04-01,2,2
2,2013-04-01,2,3
3,2013-04-01,2,4
4,2013-04-01,2,5
5,2013-04-01,2,6
6,2013-04-01,2,7
7,2013-04-01,2,8
8,2013-04-01,2,9
9,2013-04-01,2,10


### sampleSubmission.csv File

In [124]:
data_sampleSubmission = pd.read_csv('sampleSubmission.csv')
data_sampleSubmission

,id,units
0,2_1_2013-04-01,0
1,2_2_2013-04-01,0
2,2_3_2013-04-01,0
3,2_4_2013-04-01,0
4,2_5_2013-04-01,0
5,2_6_2013-04-01,0
6,2_7_2013-04-01,0
7,2_8_2013-04-01,0
8,2_9_2013-04-01,0
9,2_10_2013-04-01,0
